In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tensorboardX import SummaryWriter

from utils import tabularModel, titanicDataset

from ray import tune
from ray.tune.schedulers import ASHAScheduler
from ray.tune.suggest.bayesopt import BayesOptSearch

import time
%matplotlib inline

In [2]:
def set_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.manual_seed(seed)
    np.random.seed(seed)
set_seed(0)

In [3]:
train_data = pd.read_csv(os.path.join('data', 'titanic', 'train.csv'))

In [4]:
train_data.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [11]:
def train(config):
    batch_size = int(config['batch_size'])
    num_embedding = int(config['num_embedding'])
    learning_rate = config['learning_rate']
    fc_size = (int(config['fc_size_0']), int(config['fc_size_1']))
    dropout = (config['dropout_0'], config['dropout_1'])
    if 'save' in list(config.keys()):
        writer = SummaryWriter('tmp/batchsize-{}-lr-{}-embed-{}'.format(batch_size, learning_rate, num_embedding))
    dataset = titanicDataset(os.path.join('/ldap_home', 'chang.liao', '4483A3', 'data', 'titanic', 'train.csv'))
    train_dataset, valid_dataset = torch.utils.data.random_split(dataset,
                                                                 [int(0.8 * len(dataset)),
                                                                  len(dataset) - int(0.8 * len(dataset))])
    train_dataloader = DataLoader(train_dataset,
                                  batch_size=batch_size, shuffle=True, num_workers=4)
    valid_dataloader = DataLoader(valid_dataset,
                                 batch_size=len(valid_dataset), shuffle=False, num_workers=4)
    categorical_columns = ['Pclass', 'Sex', 'Embarked', 'binned_family']
    categories = [3, 2, 3, 4]
    continuous_columns = ['Fare', 'Age']
    embedding_size = list(zip(categories, [num_embedding] * 4))
    model = tabularModel(embedding_size, categorical_columns, continuous_columns, fc_size, dropout).double()
    optim = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=0.95)
    lossfunc = nn.CrossEntropyLoss()
    num_epoch = 300
    best_accu = 0
    cnt = 0
    model.train()
    for epoch in range(num_epoch):
        for i, (x, y) in enumerate(train_dataloader):
            optim.zero_grad()
            pred = model(x)
            loss = lossfunc(pred, y.squeeze().long())
            loss.backward()
            optim.step()

        with torch.no_grad():
            model.eval()
            x, y = next(iter(valid_dataloader))
            pred = model(x)
            valid_loss = lossfunc(pred, y.squeeze().long())
            accu = (pred.argmax(-1) == y.squeeze()).sum().item() / len(valid_dataset)
        if 'save' in list(config.keys()):
            print("Epoch: {} | Train loss: {:.4f} | Valid loss: {:.4f} | Valid Accu: {:.4f}".format(
                epoch, loss.item(), valid_loss.item(), accu
            ))
        
            writer.add_scalars('loss', {'train loss': loss.item(),
                                    'valid loss': valid_loss.item()
                                    }, epoch * (len(train_dataset) // batch_size) + i)
            writer.add_scalar('valid accu', accu,  epoch * (len(train_dataset) // batch_size) + i)
        model.train()
        # print("\n {:.4f} {:.4f} {} \n".format(1.02 * best_accu, accu, cnt))
        if accu >= best_accu:
            cnt = 0
            best_accu = accu
            if 'save' in list(config.keys()):
                torch.save(model.state_dict(), os.path.join('model', 'titanic',
                                                        'model-{}.pt'.format(epoch)))
        else:
            cnt += 1

        if cnt == 50:
            # print("Early stopped!")
            break
    return best_accu

In [6]:
# def raytune_trainable(config, checkpoint_dir=None):
#     tune.report(accu=train(config))

In [7]:
# search_space = {
#     'batch_size': (2, 64),
#     'num_embedding': (4, 256),
#     'learning_rate': (1e-4, 0.1),
#     'fc_size_0': (64, 256),
#     'fc_size_1': (8, 128),
#     'dropout_0': (0.1, 0.8),
#     'dropout_1': (0.1, 0.8)
# }

In [10]:
# algo = BayesOptSearch(space=search_space, metric='accu', mode='max', random_search_steps=5)
# start = time.time()
# analysis = tune.run(raytune_trainable, search_alg=algo, num_samples=100, 
#                     # checkpoint_freq=4,
#                     # checkpoint_at_end=True,
#                     local_dir=os.curdir,
#                     resources_per_trial={'cpu': 18},
#                     scheduler = ASHAScheduler(
#                         metric='accu',
#                         mode='max'),
#                     raise_on_failed_trial=False
#                    )
# end = time.time()
# print(end-start)

2020-11-11 16:47:20,654	INFO services.py:1092 -- View the Ray dashboard at http://127.0.0.1:8265
2020-11-11 16:47:22,888	WARNING tune.py:401 -- Tune detects GPUs, but no trials are using GPUs. To enable trials to use GPUs, set tune.run(resources_per_trial={'gpu': 1}...) which allows Tune to expose 1 GPU to each trial. You can also override `Trainable.default_resource_request` if using the Trainable API.
2020-11-11 16:47:22,895	INFO logger.py:201 -- pip install 'ray[tune]' to see TensorBoard files.
2020-11-11 16:47:22,896	WARNING logger.py:343 -- Could not instantiate TBXLogger: No module named 'tensorboardX'.


Trial name,status,loc,batch_size,dropout_0,dropout_1,fc_size_0,fc_size_1,learning_rate,num_embedding
raytune_trainable_8408eb2c,RUNNING,,25.2215,0.7655,0.612396,178.942,26.7222,0.0156839,18.6371


Result for raytune_trainable_840cd0c0:
  accu: 0.5698324022346368
  date: 2020-11-11_16-48-36
  done: false
  experiment_id: f7debc176c98470c98519ac85adf3288
  experiment_tag: 2_batch_size=55.703,dropout_0=0.52078,dropout_1=0.59565,fc_size_0=67.952,fc_size_1=124.39,learning_rate=0.083261,num_embedding=57.509
  hostname: shopee-ds-ip-10-128-148-194.idata-server.shopee.io
  iterations_since_restore: 1
  node_ip: 10.128.148.194
  pid: 55197
  time_since_restore: 72.78842234611511
  time_this_iter_s: 72.78842234611511
  time_total_s: 72.78842234611511
  timestamp: 1605084516
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 840cd0c0
  


Trial name,status,loc,batch_size,dropout_0,dropout_1,fc_size_0,fc_size_1,learning_rate,num_embedding,iter,total time (s),accu
raytune_trainable_8408eb2c,RUNNING,,25.2215,0.7655,0.612396,178.942,26.7222,0.0156839,18.6371,,,
raytune_trainable_840cd0c0,RUNNING,10.128.148.194:55197,55.7029,0.520781,0.595651,67.9522,124.389,0.083261,57.5095,1,72.7884,0.569832
raytune_trainable_840ddefc,RUNNING,,13.2731,0.228383,0.31297,164.753,59.8334,0.0291938,158.187,,,
raytune_trainable_840ee554,PENDING,,10.6486,0.304501,0.356453,151.565,102.221,0.0200474,133.587,,,


Result for raytune_trainable_8408eb2c:
  accu: 0.6201117318435754
  date: 2020-11-11_16-49-21
  done: false
  experiment_id: 076ccc0ea91f4db7a0792935f5da1571
  experiment_tag: 1_batch_size=25.221,dropout_0=0.7655,dropout_1=0.6124,fc_size_0=178.94,fc_size_1=26.722,learning_rate=0.015684,num_embedding=18.637
  hostname: shopee-ds-ip-10-128-148-194.idata-server.shopee.io
  iterations_since_restore: 1
  node_ip: 10.128.148.194
  pid: 55187
  time_since_restore: 118.1078028678894
  time_this_iter_s: 118.1078028678894
  time_total_s: 118.1078028678894
  timestamp: 1605084561
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 8408eb2c
  


...

Trial name,status,loc,batch_size,dropout_0,dropout_1,fc_size_0,fc_size_1,learning_rate,num_embedding,iter,total time (s),accu
raytune_trainable_2dfd799a,RUNNING,,2.14535,0.436243,0.181375,237.006,62.1063,0.033133,192.972,,,
raytune_trainable_04c33ba2,TERMINATED,,59.5251,0.326596,0.433798,69.8403,62.2751,0.0946879,169.337,1,72.7251,0.687151
raytune_trainable_084bb7dc,TERMINATED,,5.88995,0.789094,0.261555,78.0629,126.921,0.0877948,227.159,1,604.908,0.603352
raytune_trainable_09efe534,TERMINATED,,30.2286,0.487874,0.273135,239.769,27.0669,0.077835,75.2823,1,118.281,0.648045
raytune_trainable_118064ac,TERMINATED,,57.3269,0.330678,0.607394,200.136,31.7699,0.0715476,16.9448,1,70.8299,0.575419
raytune_trainable_19691b52,TERMINATED,,4.78717,0.207571,0.406274,186.28,22.9175,0.0246863,18.6113,1,647.158,0.608939
raytune_trainable_1a74137a,TERMINATED,,50.1994,0.193136,0.577967,183.281,50.282,0.00652167,160.365,1,16.6594,0.821229
raytune_trainable_1cdd842c,TERMINATED,,27.5084,0.777045,0.283553,89.1599,8.34247,0.094363,138.593,1,106.188,0.575419
raytune_trainable_1d36f818,TERMINATED,,43.1038,0.550045,0.747026,151.084,86.7946,0.0419965,217.282,1,123.424,0.653631
raytune_trainable_1dc5e520,TERMINATED,,45.4708,0.710266,0.593451,135.23,76.5181,0.0220258,57.4247,1,13.2735,0.748603


Result for raytune_trainable_2dfd799a:
  accu: 0.6703910614525139
  date: 2020-11-11_18-30-36
  done: true
  experiment_id: b0988617d743468e9a9b970ad042a73e
  experiment_tag: 97_batch_size=2.1453,dropout_0=0.43624,dropout_1=0.18138,fc_size_0=237.01,fc_size_1=62.106,learning_rate=0.033133,num_embedding=192.97
  hostname: shopee-ds-ip-10-128-148-194.idata-server.shopee.io
  iterations_since_restore: 1
  node_ip: 10.128.148.194
  pid: 52510
  time_since_restore: 1594.4100363254547
  time_this_iter_s: 1594.4100363254547
  time_total_s: 1594.4100363254547
  timestamp: 1605090636
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 2dfd799a
  


Trial name,status,loc,batch_size,dropout_0,dropout_1,fc_size_0,fc_size_1,learning_rate,num_embedding,iter,total time (s),accu
raytune_trainable_04c33ba2,TERMINATED,,59.5251,0.326596,0.433798,69.8403,62.2751,0.0946879,169.337,1,72.7251,0.687151
raytune_trainable_084bb7dc,TERMINATED,,5.88995,0.789094,0.261555,78.0629,126.921,0.0877948,227.159,1,604.908,0.603352
raytune_trainable_09efe534,TERMINATED,,30.2286,0.487874,0.273135,239.769,27.0669,0.077835,75.2823,1,118.281,0.648045
raytune_trainable_118064ac,TERMINATED,,57.3269,0.330678,0.607394,200.136,31.7699,0.0715476,16.9448,1,70.8299,0.575419
raytune_trainable_19691b52,TERMINATED,,4.78717,0.207571,0.406274,186.28,22.9175,0.0246863,18.6113,1,647.158,0.608939
raytune_trainable_1a74137a,TERMINATED,,50.1994,0.193136,0.577967,183.281,50.282,0.00652167,160.365,1,16.6594,0.821229
raytune_trainable_1cdd842c,TERMINATED,,27.5084,0.777045,0.283553,89.1599,8.34247,0.094363,138.593,1,106.188,0.575419
raytune_trainable_1d36f818,TERMINATED,,43.1038,0.550045,0.747026,151.084,86.7946,0.0419965,217.282,1,123.424,0.653631
raytune_trainable_1dc5e520,TERMINATED,,45.4708,0.710266,0.593451,135.23,76.5181,0.0220258,57.4247,1,13.2735,0.748603
raytune_trainable_1f3f5c10,TERMINATED,,34.8869,0.541266,0.792842,115.977,87.9149,0.0827367,152.703,1,109.273,0.597765


Trial name,status,loc,batch_size,dropout_0,dropout_1,fc_size_0,fc_size_1,learning_rate,num_embedding,iter,total time (s),accu
raytune_trainable_8408eb2c,TERMINATED,,25.2215,0.7655,0.612396,178.942,26.7222,0.0156839,18.6371,1,118.108,0.620112
raytune_trainable_840cd0c0,TERMINATED,,55.7029,0.520781,0.595651,67.9522,124.389,0.083261,57.5095,1,72.7884,0.569832
raytune_trainable_840ddefc,TERMINATED,,13.2731,0.228383,0.31297,164.753,59.8334,0.0291938,158.187,1,263.825,0.592179
raytune_trainable_840ee554,TERMINATED,,10.6486,0.304501,0.356453,151.565,102.221,0.0200474,133.587,1,51.8132,0.703911
raytune_trainable_afeecf36,TERMINATED,,38.7297,0.132515,0.525281,96.7406,15.8062,0.0948937,247.339,1,121.8,0.625698
raytune_trainable_21cd3fca,TERMINATED,,11.3921,0.369202,0.130242,154.1,98.37,0.0790183,129.745,1,302.172,0.620112
raytune_trainable_21d5d496,TERMINATED,,35.2193,0.393282,0.459743,91.6478,12.8588,0.00641037,249.152,1,21.3264,0.793296
raytune_trainable_21dc1de2,TERMINATED,,5.88289,0.790241,0.427749,160.339,103.568,0.046476,131.438,1,648.148,0.687151
raytune_trainable_21e2bbf2,TERMINATED,,36.1205,0.389657,0.731134,101.514,14.312,0.0475347,241.336,1,14.8296,0.698324
raytune_trainable_2f20b51c,TERMINATED,,36.4767,0.732621,0.563282,108.258,123.934,0.039675,10.9711,1,83.7698,0.620112


2020-11-11 18:30:37,288	ERROR tune.py:436 -- Trials did not complete: [raytune_trainable_772c6608, raytune_trainable_f50bb74c]
2020-11-11 18:30:37,289	INFO tune.py:439 -- Total run time: 6197.29 seconds (6194.18 seconds for the tuning loop).
6197.54792881012


In [12]:
# outcome = analysis.dataframe()

In [13]:
# idx = outcome.accu.argmax()

In [14]:
# os.listdir(outcome.loc[idx, :].logdir)

['params.pkl', 'progress.csv', 'result.json', 'params.json']

In [15]:
# import json

In [16]:
# with open(os.path.join(outcome.loc[idx, :].logdir, 'params.json'), 'r') as fin:
#     config = json.load(fin)

In [17]:
# config['save'] = True
# config

{'batch_size': 43.72186742201144,
 'dropout_0': 0.4477666170489303,
 'dropout_1': 0.4874199109505203,
 'fc_size_0': 115.73881337682644,
 'fc_size_1': 58.49153573597393,
 'learning_rate': 0.024569201171483243,
 'num_embedding': 253.7724832186956,
 'save': True}

In [18]:
# with open(os.path.join(outcome.loc[idx, :].logdir, 'result.json'), 'r') as fin:
#     result = json.load(fin)
# result

{'accu': 0.8379888268156425,
 'time_this_iter_s': 15.517733097076416,
 'done': False,
 'timesteps_total': None,
 'episodes_total': None,
 'training_iteration': 1,
 'experiment_id': '8d6d6c10799b4616bd4528f95fb75d92',
 'date': '2020-11-11_17-03-24',
 'timestamp': 1605085404,
 'time_total_s': 15.517733097076416,
 'pid': 55208,
 'hostname': 'shopee-ds-ip-10-128-148-194.idata-server.shopee.io',
 'node_ip': '10.128.148.194',
 'config': {'batch_size': 43.72186742201144,
  'dropout_0': 0.4477666170489303,
  'dropout_1': 0.4874199109505203,
  'fc_size_0': 115.73881337682644,
  'fc_size_1': 58.49153573597393,
  'learning_rate': 0.024569201171483243,
  'num_embedding': 253.7724832186956},
 'time_since_restore': 15.517733097076416,
 'timesteps_since_restore': 0,
 'iterations_since_restore': 1,
 'trial_id': 'ae8bb88c',
 'experiment_tag': '22_batch_size=43.722,dropout_0=0.44777,dropout_1=0.48742,fc_size_0=115.74,fc_size_1=58.492,learning_rate=0.024569,num_embedding=253.77'}

In [24]:
# outcome[outcome['done'] == True].accu.max()

0.7430167597765364

In [48]:
config = {
    'batch_size': 16,
    'dropout_0': 0.6,
    'dropout_1': 0.5,
    'fc_size_0': 256,
    'fc_size_1': 48,
    'learning_rate': 1e-4,
    'num_embedding': 128,
    'save': True
 }

In [49]:
train(config)

Epoch: 0 | Train loss: 0.5553 | Valid loss: 0.6872 | Valid Accu: 0.5419
Epoch: 1 | Train loss: 0.5551 | Valid loss: 0.6436 | Valid Accu: 0.7542
Epoch: 2 | Train loss: 0.6668 | Valid loss: 0.6363 | Valid Accu: 0.7207
Epoch: 3 | Train loss: 0.6180 | Valid loss: 0.6246 | Valid Accu: 0.6927
Epoch: 4 | Train loss: 0.6303 | Valid loss: 0.6293 | Valid Accu: 0.7374
Epoch: 5 | Train loss: 0.5842 | Valid loss: 0.5867 | Valid Accu: 0.7654
Epoch: 6 | Train loss: 0.6579 | Valid loss: 0.6048 | Valid Accu: 0.7598
Epoch: 7 | Train loss: 0.7298 | Valid loss: 0.5978 | Valid Accu: 0.7542
Epoch: 8 | Train loss: 0.8604 | Valid loss: 0.5826 | Valid Accu: 0.7542
Epoch: 9 | Train loss: 0.5450 | Valid loss: 0.5810 | Valid Accu: 0.7542
Epoch: 10 | Train loss: 0.4868 | Valid loss: 0.6159 | Valid Accu: 0.7430
Epoch: 11 | Train loss: 0.7254 | Valid loss: 0.6056 | Valid Accu: 0.7542
Epoch: 12 | Train loss: 0.7897 | Valid loss: 0.5984 | Valid Accu: 0.7542
Epoch: 13 | Train loss: 0.5861 | Valid loss: 0.6228 | Valid A

0.8156424581005587

In [29]:
def eval(config):
    batch_size = int(config['batch_size'])
    num_embedding = int(config['num_embedding'])
    learning_rate = config['learning_rate']
    fc_size = (int(config['fc_size_0']), int(config['fc_size_1']))
    dropout = (config['dropout_0'], config['dropout_1'])
    dataset = titanicDataset(os.path.join('/ldap_home', 'chang.liao', '4483A3', 'data', 'titanic', 'test.csv'), train=False)
    dataloader = DataLoader(dataset, batch_size=len(dataset), num_workers=4, shuffle=False)
    categorical_columns = ['Pclass', 'Sex', 'Embarked', 'binned_family']
    categories = [3, 2, 3, 4]
    continuous_columns = ['Fare', 'Age']
    embedding_size = list(zip(categories, [num_embedding] * 4))
    model = tabularModel(embedding_size, categorical_columns, continuous_columns, fc_size, dropout).double()
    model.load_state_dict(torch.load(os.path.join('model', 'titanic', config['PATH'])))
    model.eval()
    with torch.no_grad():
        x = next(iter(dataloader))
        pred = model(x)
        out = pred.argmax(-1)
    return out

In [30]:
eval_config = config.copy()
eval_config['PATH'] = 'model-69.pt'

In [31]:
out = eval(eval_config)

In [32]:
out.numpy()


array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,